# Sagemaker and Seldon Core Scikit-learn Example

 * Train a model on AWS Sagemaker
 * Deploy locally on Seldon Core
 
 This tutorial assumes you are already familiar with Sagemaker and have an AWS account you can use.
 
 Dependencies
 
  * AWS CLI
  * Docker
  * Git
  * Minikube

## Train Scikit-learn Iris Model

We will use the Sagemaker example notebook [Iris Training and Prediction with Sagemaker Scikit-learn](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/scikit_learn_iris/Scikit-learn%20Estimator%20Example%20With%20Batch%20Transform.ipynb)

  1. Run this notebook inside Sagemaker but change the training step to include an ```output_path``` parameter with value an S3 bucket you have created. For example ```output_path="s3://<my-bucket>/<my-path>``` as shown below:
     * <img src="sagemaker-scikit-iris-train.png"/>
  1. Once trained you should find the model saved to the location you specified. You will need to use this S3 location later when deploying locally.

## Create Sagemaker SKLearn Image

To run your model locally you will need to create the sagemaker sklearn image

In [ ]:
!git clone https://github.com/aws/sagemaker-scikit-learn-container.git

Build the base container.

In [ ]:
!cd sagemaker-scikit-learn-container && \
    docker build -t sklearn-base:0.20.0-cpu-py3 -f docker/0.20.0/base/Dockerfile.cpu --build-arg py_version=3 .

In [ ]:
!cd sagemaker-scikit-learn-container && python setup.py bdist_wheel

In [ ]:
!cd sagemaker-scikit-learn-container && docker build -t sklearn-final:0.20.0-cpu-py3 -f docker/0.20.0/final/Dockerfile.cpu --build-arg py_version=3 .

**Specify the location of the model.tar.gz that AWS Sagemaker created on S3** An example is shown below.

In [ ]:
%env SAGEMAKER_MODEL_DIRECTORY=s3://seldon-sagemaker-testing/scikit_learn_iris/sagemaker-scikit-learn-2019-01-04-19-26-40-470/output/model.tar.gz

## Prepare Prediction Code

We will create the methods required to load and predict against our trained model as a module that can be loaded by the Sagemaker scikit-learn-container image we created.

In [ ]:
!pygmentize scikit_learn_iris/scikit_learn_iris.py

Package the code as a tar.gz file

In [ ]:
!cd scikit_learn_iris && tar -cvf ../scikit_learn_iris_code.tar .
!gzip -f scikit_learn_iris_code.tar

**Set the path to store the code**

In [ ]:
%env SAGEMAKER_SUBMIT_DIRECTORY=s3://seldon-sagemaker-testing/scikit_learn_iris_code.tar.gz

In [ ]:
!aws s3 cp scikit_learn_iris_code.tar.gz ${SAGEMAKER_SUBMIT_DIRECTORY}

We can now do a local test with Docker. To make this work you will need to have your aws credentials in ~/.aws/config.

In [ ]:
!docker run -d --rm -p 8080:8080 \
        -e SAGEMAKER_MODEL_DIRECTORY=${SAGEMAKER_MODEL_DIRECTORY} \
        -e SAGEMAKER_SUBMIT_DIRECTORY=${SAGEMAKER_SUBMIT_DIRECTORY} \
        -e SAGEMAKER_PROGRAM="scikit_learn_iris.py" \
        -v ~/.aws:/root/.aws \
        --name "scikit_predictor" \
        sklearn-final:0.20.0-cpu-py3 serve

Test the running image with a prediction call

In [ ]:
!curl 0.0.0.0:8080/invocations -d '1.0,2.0,3.0,4.0' -H "Content-Type: text/csv"

In [ ]:
!docker rm -f scikit_predictor

## Test with Seldon-Core in Minikube

In [ ]:
!minikube start --memory 4096

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

Recreate the Sagemaker scikit-learn base and final containers inside Minikube. In a production scenario you would build these images and push to your DockerHub or private repo.

In [ ]:
!eval $(minikube docker-env) && \
    cd sagemaker-scikit-learn-container && \
    docker build -t sklearn-base:0.20.0-cpu-py3 -f docker/0.20.0/base/Dockerfile.cpu --build-arg py_version=3 .

In [ ]:
!eval $(minikube docker-env) && \
    cd sagemaker-scikit-learn-container && \
    docker build -t sklearn-final:0.20.0-cpu-py3 -f docker/0.20.0/final/Dockerfile.cpu --build-arg py_version=3 .

## Create AWS Credential Secret from Template


In [ ]:
!cp aws-config-secret.yaml.tmpl aws-config-secret.yaml

Edit aws-config-secret.yaml and fill in the base64 encoded values, for example you can run on linux:

 ``` echo -n <my_aws_access_id> | base64```

Take the value and place into the file.

Once completed you can create the secret via ```kubectl```

In [ ]:
!kubectl create -f aws-config-secret.yaml

## Create Seldon Deployment from Template

We will use the environment variables you set above to fill in the details for the Seldon Deployment so the Sagemake image container knows where to download your code.

In [ ]:
!cat scikit_learn_iris_deployment.json.tmpl | sed s#{SAGEMAKER_MODEL_DIRECTORY}#${SAGEMAKER_MODEL_DIRECTORY}# | sed s#{SAGEMAKER_SUBMIT_DIRECTORY}#${SAGEMAKER_SUBMIT_DIRECTORY}# > scikit_learn_iris_deployment.json

In [ ]:
!kubectl create -f scikit_learn_iris_deployment.json

## Test Predcitive Endpoint

In [ ]:
!seldon-core-api-tester contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

## Teardown

In [ ]:
!minikube delete